<a href="https://colab.research.google.com/github/howsworkzco/avasgpt.github.io/blob/main/avasgpt_nano_vocab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import torch
import os
checkpoint_dir = '/content/drive/My Drive/AvasGPT_Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    print(f"Created directory: {checkpoint_dir}")
else:
    print("Checkpoint directory found!")

# Check if a GPU is available (Crucial for "Genius" speed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"AvasGPT is running on: {device}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checkpoint directory found!
AvasGPT is running on: cuda


In [ ]:
# tokenization
from datasets import load_dataset
import tiktoken
import numpy as np

# 1. Load the dataset in streaming mode
# This allows us to train on 38GB of data without downloading it all
ds = load_dataset("Skylion007/openwebtext", split="train", streaming=True)

# 2. Setup the Tokenizer (using the GPT-2 encoding)
enc = tiktoken.get_encoding("gpt2")

def get_batch(dataset_iterator, batch_size, block_size):
    """
    Helper function to grab a chunk of text and format it for the model.
    """
    # Grab a few examples from the stream
    texts = [next(dataset_iterator)['text'] for _ in range(batch_size)]

    # Tokenize them all
    all_tokens = []
    for t in texts:
        all_tokens.extend(enc.encode_ordinary(t))

    # Ensure we have enough tokens for the batch
    if len(all_tokens) < (batch_size * block_size + 1):
        # Recursively get more if needed (rare with large texts)
        return get_batch(dataset_iterator, batch_size, block_size)

    # Convert to PyTorch tensors
    data = torch.tensor(all_tokens[:batch_size * block_size + 1], dtype=torch.long)
    x = data[:-1].view(batch_size, block_size)
    y = data[1:].view(batch_size, block_size)

    return x.to(device), y.to(device)

# Initialize the stream
data_iter = iter(ds)

# Test it
X, Y = get_batch(data_iter, batch_size=4, block_size=8)
print("Data looks good! Shape of input:", X.shape)
print("Sample tokens for AvasGPT to learn:", X[0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/80 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/80 [00:00<?, ?it/s]

Data looks good! Shape of input: torch.Size([4, 8])
Sample tokens for AvasGPT to learn: tensor([13924,    12,   559,    12, 35784,    11, 25051,   357],
       device='cuda:0')


In [ ]:
# Documentation / Guide
'''
In order to continue training AvasGPT,run numbers in order:
0. RUN THIS FIRST EVERY TIME
1. model architecture
3. test generate text
2. training (loop)
'''


In [ ]:
# 1. model architecture
import torch
import torch.nn as nn
from torch.nn import functional as F

class SimpleGPT(nn.Module):
    def __init__(self, vocab_size, n_embd=384, n_head=6, n_layer=6, block_size=256, dropout=0.2):
        super().__init__()
        self.block_size = block_size
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.dropout = nn.Dropout(dropout)

        # We use EncoderLayers but treat them as Decoders by using a causal mask
        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=n_embd,
                nhead=n_head,
                dim_feedforward=4*n_embd,
                dropout=dropout,
                batch_first=True,
                norm_first=True # Better for stability
            ) for _ in range(n_layer)
        ])

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # Token and position embeddings
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = self.dropout(tok_emb + pos_emb)

        # CREATE CAUSAL MASK (The "No Cheating" Mask)
        # This prevents token at index 'i' from seeing token 'i+1'
        mask = torch.full((T, T), float('-inf'), device=device)
        mask = torch.triu(mask, diagonal=1)

        for block in self.blocks:
            # is_causal=True is a PyTorch optimization for T4/A100 GPUs
            x = block(x, src_mask=mask, is_causal=True)

        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

# Initialize
model = SimpleGPT(vocab_size=enc.n_vocab).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

# Print parameter count (just to see how big your AI is!)
params = sum(p.numel() for p in model.parameters())/1e6
print(f"AvasGPT initialized with {params:.2f}M parameters.")

AvasGPT initialized with 49.39M parameters.


In [ ]:
# 0. RUN THIS FIRST EVERY TIME
import os
import torch
import tiktoken
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Setup Device & Tokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
enc = tiktoken.get_encoding("gpt2")
checkpoint_dir = '/content/drive/My Drive/AvasGPT_Checkpoints'

print(f"✅ Environment Ready! Running on: {device}")
print(f"✅ Tokenizer 'enc' is defined.")


Mounted at /content/drive
✅ Environment Ready! Running on: cuda
✅ Tokenizer 'enc' is defined.


In [ ]:
# 2. training (loop)
import os
import math

# --- Configuration ---
max_iters = 10000 # Let it run longer
eval_interval = 500
batch_size = 32   # Doubled for stability if T4 memory allows
block_size = 256
learning_rate = 3e-4
checkpoint_path = os.path.join(checkpoint_dir, 'avas_latest.pt')

# --- Learning Rate Scheduler Logic ---
def get_lr(it):
    # 1) Linear warmup for 500 steps
    if it < 500: return learning_rate * it / 500
    # 2) If it > max_iters, return min learning rate
    if it > max_iters: return learning_rate * 0.1
    # 3) In between, use cosine decay
    decay_ratio = (it - 500) / (max_iters - 500)
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return 0.1 * learning_rate + coeff * learning_rate

# --- Resume Checkpoint ---
start_iter = 0
if os.path.exists(checkpoint_path):
    print(f"Resuming AvasGPT...")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_iter = checkpoint['step']

print("Training with Cosine Decay...")

for i in range(start_iter, max_iters):
    # Set current learning rate
    lr = get_lr(i)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    x, y = get_batch(data_iter, batch_size, block_size)
    logits, loss = model(x, y)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # Clip gradients to prevent "exploding" (common in Transformers)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

    if i % eval_interval == 0:
        print(f"Step {i} | Loss {loss.item():.4f} | LR {lr:.2e}")
        torch.save({'step': i, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, checkpoint_path)
        print(f"Sample: {generate_text(model, 'Artificial Intelligence is', max_new_tokens=40)}")



Resuming AvasGPT...
Training with Cosine Decay...


NameError: name 'get_batch' is not defined

In [ ]:
# 3. test generate text
def generate_text(model, prompt, max_new_tokens=100, temperature=0.8, top_k=50):
    model.eval()
    idx = torch.tensor(enc.encode(prompt), dtype=torch.long, device=device).unsqueeze(0)

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / temperature

        # TOP-K SAMPLING: Filter out the "junk" words
        v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
        logits[logits < v[:, [-1]]] = -float('Inf')

        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)

    model.train()
    return enc.decode(idx.tolist())


In [ ]:
## 🤖 Chat with AvasGPT
# --- 1. Load the Brain from Drive ---
checkpoint_path = os.path.join(checkpoint_dir, 'avas_latest.pt')

if os.path.exists(checkpoint_path):
    # Load the 565MB file into the model
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval() # Important: Tells the AI to STOP learning and START talking
    print("✅ AvasGPT is awake and ready to chat!")
else:
    print("❌ Could not find 'avas_latest.pt' in your Drive. Run the training cell first!")

# --- 2. The Interaction Loop ---
# You can change the 'prompt' to anything you want!
prompt = "The secret of the eternal child is"
response = generate_text(model, prompt, max_new_tokens=100, temperature=0.8)

print("\n--- AvasGPT Response ---")
print(response)
print("------------------------")


NameError: name 'checkpoint_dir' is not defined

In [ ]:
!pip install tiktoken datasets


In [ ]:
import os
import glob

# Keep only the last two checkpoints to save about 4GB of space
all_checkpoints = sorted(glob.glob(os.path.join(checkpoint_dir, 'avasgpt_step_*.pt')))
if len(all_checkpoints) > 2:
    for old_file in all_checkpoints[:-2]: # Keep the 2 newest ones
        os.remove(old_file)
        print(f"Deleted old version to save 565MB: {os.path.basename(old_file)}")


Deleted old version to save 565MB: avasgpt_step_0.pt
Deleted old version to save 565MB: avasgpt_step_1000.pt
Deleted old version to save 565MB: avasgpt_step_1500.pt
Deleted old version to save 565MB: avasgpt_step_2000.pt
Deleted old version to save 565MB: avasgpt_step_2500.pt
Deleted old version to save 565MB: avasgpt_step_3000.pt
Deleted old version to save 565MB: avasgpt_step_3500.pt
Deleted old version to save 565MB: avasgpt_step_4000.pt


In [ ]:
# original number 2
'''import time

max_iters = 5000
eval_interval = 500
batch_size = 16 # Adjust based on T4 memory
block_size = 64

print("Starting training...")
for i in range(max_iters):
    # 1. Get a batch of data
    x, y = get_batch(data_iter, batch_size, block_size)

    # 2. Forward pass & Loss calculation
    logits, loss = model(x, y)

    # 3. Backward pass (Backpropagation)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # 4. Progress check and Saving
    if i % eval_interval == 0:
        print(f"Step {i}: Loss {loss.item():.4f}")
        # Save checkpoint to your Google Drive
        checkpoint_path = os.path.join(checkpoint_dir, f'avasgpt_step_{i}.pt')
        torch.save({
            'step': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item(),
        }, checkpoint_path)
        print(f"Saved checkpoint to: {checkpoint_path}")
        '''
        '''original v2import os

# --- Configuration ---
max_iters = 5000
eval_interval = 500
batch_size = 16
block_size = 256 # Increased for better memory/context
checkpoint_path = os.path.join(checkpoint_dir, 'avas_latest.pt')

# --- 1. Load Checkpoint if it exists ---
start_iter = 0
if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint from {checkpoint_path}...")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_iter = checkpoint['step']
    print(f"Resuming from step {start_iter}")

# --- 2. Training Loop ---
model.train()
print("Starting training...")

for i in range(start_iter, max_iters):
    # Get data
    x, y = get_batch(data_iter, batch_size, block_size)

    # Forward & Backward
    logits, loss = model(x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # Progress check
    if i % eval_interval == 0 or i == max_iters - 1:
        print(f"Step {i}: Loss {loss.item():.4f}")

        # Auto-Save to Google Drive
        torch.save({
            'step': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, checkpoint_path)

        # Test Sample
        print("--- AvasGPT says: ---")
        print(generate_text(model, "The secret to AI is", max_new_tokens=30))
        print("---------------------")
'''